In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image

# ------------------------
# README
# 훈련된 .pth 모델을 불러와 단일 이미지에 대한 예측을 수행하는 코드입니다.
# .pth 모델들도 깃허브에 있습니다
# ------------------------
# 사용 방법
# 1. 훈련된 모델 불러오기
# 2. 단일 이미지 로드 및 예측 부분에 이미지 경로 수정


# ------------------------
# 1. 훈련된 모델 불러오기
# ------------------------
model_path = "/Users//Desktop/AI/ResNet Models/resnet50(aug)_3/resnet50_augmented_model.pth"  # 저장된 모델 경로
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")  # 맥 MPS 사용

# ResNet50 모델 불러오기
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
num_features = model.fc.in_features  # 마지막 레이어의 입력 노드 수
num_classes = 12  # 클래스 개수 (데이터셋에 맞게 변경)

# 마지막 레이어를 학습된 모델에 맞게 수정
model.fc = nn.Linear(num_features, num_classes)

# 저장된 가중치 불러오기
model.load_state_dict(torch.load(model_path, map_location=device))

# 모델을 GPU(MPS)로 이동 및 평가 모드 설정
model.to(device)
model.eval()  # 평가 모드 (Dropout, BatchNorm 비활성화)

print("✅ 훈련된 모델이 성공적으로 로드되었습니다!")

# ------------------------
# 2. 이미지 전처리 정의 (훈련 시 적용했던 정규화와 동일해야 함)
# ------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조정 (224x224)
    transforms.ToTensor(),  # 텐서 변환
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ResNet 학습 시 사용한 정규화 적용
])

# ------------------------
# 3. 단일 이미지 로드 및 예측
# ------------------------
image_path = "/Users/Downloads/coco_sample2.jpg"  # 테스트할 이미지 경로 (수정 필요)

# 이미지 열기 및 전처리 적용
image = Image.open(image_path).convert("RGB")  # 흑백 이미지를 방지하기 위해 RGB 변환
image = transform(image)  # 변환 적용
image = image.unsqueeze(0)  # 배치 차원 추가 (모델 입력 크기 맞추기)

# 이미지를 GPU(MPS)로 이동
image = image.to(device)

# 모델 예측 수행
with torch.no_grad():  # 그래디언트 계산 비활성화 (메모리 절약)
    outputs = model(image)  # 순전파 수행
    _, predicted_class = outputs.max(1)  # 가장 높은 확률을 가진 클래스 예측

# ------------------------
# 4. 예측 결과 출력 (클래스 인덱스를 라벨명으로 변환)
# ------------------------
class_labels = [
    "BODYLOWER", "BODYSCRATCH", "BODYSHAKE", "FEETUP", "FOOTUP",
    "HEADING", "LYING", "MOUNTING", "SIT", "STANDING",
    "TURN", "WALKRUN"
]  # 클래스 라벨 리스트 (새로운 데이터셋에 맞게 수정)


predicted_label = class_labels[predicted_class.item()]
print(f"✅ 예측된 클래스: {predicted_label}")  # 최종 예측 결과 출력
